In [1]:
from general_tools.notebook import gpu_utils
GPU = 0
gpu_utils.setup_one_gpu(GPU)
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.in_out.basics import read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.nips.helper import average_per_class, pclouds_with_zero_mean_in_unit_sphere

from sklearn.svm import LinearSVC
from general_tools.simpletons import sort_dict_by_key, sort_dict_by_val

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
def iclr_optimized_paratemers(loss):    
    if loss == 'chamfer':
        epoch_to_load = 1000
    if loss == 'emd':
        epoch_to_load = 1100
    return None, epoch_to_load    

In [5]:
n_pc_samples = 2048
put_in_usphere = True
model_net = '40'
ae_loss = 'chamfer'

In [6]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name = 'no_conv_deeper_snc_rotated2048pts_' + ae_loss

In [7]:
train_dir = osp.join(top_data_dir, experiment_name)
conf = Conf.load(osp.join(train_dir, 'configuration'))
print conf
conf.n_output = conf.n_input
conf.allow_gpu_growth = False

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm': True, 'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1, 1, 1, 1], 'n_filters': [128, 128, 256, 512], 'b_norm': True, 'strides': [1, 1, 1, 1]}
               experiment_name: no_conv_deeper_snc_rotated2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                 

In [10]:
epoch_to_load = iclr_optimized_paratemers(ae_loss)

In [12]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
last_epoch = saved_epochs[-1]
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 2000.


In [13]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net + '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = sorted([f for f in files_in_subdirs(model_net_dir, search_pattern)])

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = sorted([f for f in files_in_subdirs(model_net_dir, search_pattern)])

In [14]:
verbose = True
n_threads = 22

pc, model_names, labels = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
    pc = pclouds_with_zero_mean_in_unit_sphere(pc)

train_data = PointCloudDataSet(pc, labels=labels, init_shuffle=False)

pc_, model_names_, labels_ = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
    pc_ = pclouds_with_zero_mean_in_unit_sphere(pc_)

test_data = PointCloudDataSet(pc_, labels=labels_, init_shuffle=False)

9843 pclouds were loaded. They belong in 40 shape-classes.
2468 pclouds were loaded. They belong in 40 shape-classes.


In [ ]:
# check reconstruction by diff centerings

In [15]:
train_feed, train_latent, train_classes = ae.embedding_at_tensor(train_data, conf, tensor_name='bottleneck')
cids = classes_to_integers(int(model_net), train_classes)[1]

test_feed, test_latent, test_classes = ae.embedding_at_tensor(test_data, conf, tensor_name='bottleneck')
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [16]:
from tf_lab.data_sets.numpy_dataset import NumpyDataset
train_data = NumpyDataset([train_latent, np.array(cids)])
test_data = NumpyDataset([test_latent, np.array(cids_)])

In [38]:
import tensorflow as tf
from tflearn import is_training
from tf_lab.fundamentals.inspect import count_trainable_parameters
from tf_lab.neural_net import Neural_Net
from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only
import time

In [83]:
n_classes = 40
learning_rate = 0.001
vector_dim = 512

class Vector_CLF(Neural_Net):
    def __init__(self, name, graph=None):
#         if graph is None:
#             graph = tf.get_default_graph()
        
        Neural_Net.__init__(self, name, None)

        self.x = tf.placeholder(tf.float32, (None, vector_dim))
        self.gt = tf.placeholder(tf.int32, (None))

        self.logits = decoder_with_fc_only(self.x, [1024, 512, n_classes], b_norm=True, dropout_prob=[0.5, 0.5, 0])
        self.prediction = tf.argmax(self.logits, axis=1)

        self.correct_pred = tf.equal(self.prediction, tf.cast(self.gt, tf.int64))
        self.avg_accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))
        self._create_loss_optimizer()

        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True

        # Initializing the tensor flow variables
        self.init = tf.global_variables_initializer()

        # Launch the session
        self.sess = tf.Session(config=config)
        self.sess.run(self.init)
    
    def _create_loss_optimizer(self):        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.gt)
        self.loss = tf.reduce_mean(loss)
#         tf.summary.scalar('Classification_loss', self.loss)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

    def partial_fit(self, X, GT):
        '''Trains the model with mini-batches of input data.'''
        try:
            is_training(True, session=self.sess)
            _, loss, logits = self.sess.run((self.optimizer, self.loss, self.logits), feed_dict={self.x: X, self.gt: GT})
            is_training(False, session=self.sess)
        except Exception:
            raise
        finally:
            is_training(False, session=self.sess)
        return logits, loss

    def predict(self, X, gt_labels=None):
        feed_dict = {self.x: X}
        if gt_labels is None:
            avg_acc = tf.no_op()
        else:
            avg_acc = self.avg_accuracy
            feed_dict[self.gt] = gt_labels
        return self.sess.run((self.prediction, avg_acc), feed_dict)

    def _single_epoch_train(self, train_data, batch_size):
        n_examples = train_data.n_examples
        epoch_loss = 0.        
        n_batches = int(n_examples / batch_size)
        start_time = time.time()
        # Loop over all batches
        for _ in xrange(n_batches):
            batch_i, labels = train_data.next_batch(batch_size)
            _, loss = self.partial_fit(batch_i, labels)

            # Compute average loss
            epoch_loss += loss
        epoch_loss /= n_batches
        duration = time.time() - start_time
        return epoch_loss, duration

In [84]:
reset_tf_graph()
clf = Vector_CLF('test')

In [86]:
batch_size = 50
for i in range(200):
    print clf._single_epoch_train(train_data, batch_size)
    print clf.predict(train_data.a, train_data.b)[1], clf.predict(test_data.a, test_data.b)[1]

(0.062531923259874542, 1.7847938537597656)
0.976531 0.859805
(0.06190380633556835, 1.295799970626831)
0.976938 0.860616
(0.054138447502829916, 1.3072359561920166)
0.978055 0.867099
(0.067146938576895215, 1.219006061553955)
0.977649 0.864262
(0.059018158986724967, 1.486772060394287)
0.978258 0.859805
(0.07031675940557687, 1.3078110218048096)
0.976633 0.863452
(0.060385478445452313, 1.216399908065796)
0.982322 0.858185
(0.061291447655316821, 1.4210388660430908)
0.982322 0.867504
(0.05246154479421105, 1.3487420082092285)
0.982728 0.867504
(0.063613839183781981, 1.2750539779663086)
0.978563 0.845624
(0.062163215150762043, 1.2512822151184082)
0.978563 0.857374
(0.054963578955313112, 1.4084961414337158)
0.982322 0.865073
(0.060470356416475142, 1.3152120113372803)
0.977446 0.846839
(0.058256224364431917, 1.3509268760681152)
0.97074 0.852107
(0.056364882806297013, 1.3783769607543945)
0.979681 0.861831
(0.054810388634255754, 1.3417470455169678)
0.981713 0.858185
(0.05413667466527098, 1.19226908